Web Scraping data assignment. Toronto neighbourhood clusters.

Url = https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M of website.

In [1]:
import pandas as pd
import numpy as np
#webscraping imports
from bs4 import BeautifulSoup
import datetime
from tinydb import TinyDB, Query
import urllib3
import xlsxwriter

In [22]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /Users/DRMair/anaconda3

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:           1.22.0-pyh9f0ad1d_0 conda-forge

The following packages will be UPDATED:

    ca-certificates: 2020.1.1-0                      --> 2020.4.5.1-

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
def make_soup(url):
    http = urllib3.PoolManager()
    r = http.request("GET", url)
    return BeautifulSoup(r.data,'lxml')

In [4]:
col_names = ["Postal Code", "Borough", "Neighbourhood"]

neighbourhoods = pd.DataFrame(columns = col_names)
neighbourhoods

,Postal Code,Borough,Neighbourhood


In [5]:
soup = make_soup(url)
# htmlfile

/Users/DRMair/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [6]:
result = soup.find_all("table", class_ = "wikitable sortable")

In [7]:
result_1 = result
type(result_1[0])


# soup = BeautifulSoup(result_1[0], 'lxml') # Parse the HTML as a string

bs4.element.Tag

In [8]:
table_rows = result[0].find_all('tr')

In [9]:
list = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
#     neighbourhoods.iat[row_marker,column_marker] = column.get_text()
    list.append(row)

list.remove([])

In [10]:
for i in range(0, len(list)):
    for j in range(0,3):
        list[i][j] = list[i][j].strip('\n')

neighbourhoods = pd.DataFrame(list,columns = col_names)


In [11]:
neighbourhoods.dropna(axis = 0,how = 'any',inplace = True)

In [12]:
neighbourhoods = neighbourhoods[neighbourhoods.Borough != 'Not assigned']

In [13]:
neighbourhoods

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [16]:
neighbourhoods = neighbourhoods.reset_index()
# neighbourhoods.drop(['level_0','index'], axis = 1, inplace= True)
neighbourhoods.drop(['index'], axis = 1, inplace= True)

In [17]:
neighbourhoods

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


We have the scraped data frame with Toronto postal codes, now we will add latitudes and longitudes to the data frame using geocoder csv file.

In [19]:
lat_lng = pd.read_csv("/Users/DRMair/Downloads/Geospatial_Coordinates.csv")

In [20]:
neighbourhoods = neighbourhoods.merge(lat_lng, on = 'Postal Code' )

In [21]:
neighbourhoods

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [29]:
neighbourhoods.describe(include="all")

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
count,103,103,103,103.000000,103.000000
unique,103,10,99,NaN,NaN
top,M1M,North York,Downsview,NaN,NaN
freq,1,24,4,NaN,NaN
mean,NaN,NaN,NaN,43.704608,-79.397153
std,NaN,NaN,NaN,0.052463,0.097146
min,NaN,NaN,NaN,43.602414,-79.615819
25%,NaN,NaN,NaN,43.660567,-79.464763
50%,NaN,NaN,NaN,43.696948,-79.388790
75%,NaN,NaN,NaN,43.745320,-79.340923


On to a cluster analysis of Toronto Neighbourhoods. We will begin with k-means and branch from there with documentation along the way.

Firstly, we will create a map of Toronto and place markings on of each data point neighbourhood on this map.

In [24]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(neighbourhoods['Borough'].unique()),
        neighbourhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighbourhoods.


In [26]:
address = 'Toronto, ON'

# geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


In [28]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Borough'], neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

North York is the top Borough (according to our describe method) of the city of Toronto, so we will take a closer look.

In [38]:
NorthYork_data = neighbourhoods[neighbourhoods['Borough'] == 'North York'].reset_index(drop = True)
NorthYork_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073
5,M3C,North York,Don Mills,43.725900,-79.340923
6,M2H,North York,Hillcrest Village,43.803762,-79.363452
7,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
9,M3J,North York,"Northwood Park, York University",43.767980,-79.487262


In [44]:
address_northyork = 'North York, ON'

# geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.6563221, -79.3809161.


In [41]:
# create map of Downtowon Toronto using latitude and longitude values
map_north_york = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(NorthYork_data['Latitude'], NorthYork_data['Longitude'], NorthYork_data['Borough'], NorthYork_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_north_york) 
    
    
map_north_york

Likewise for Downtown Toronto. 

In [43]:
dt_tor_data = neighbourhoods[neighbourhoods['Borough'] == 'Downtown Toronto'].reset_index(drop = True)

In [45]:
address_northyork = 'Downtown Toronto, ON'

# geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [46]:
# create map of Downtowon Toronto using latitude and longitude values
map_dt_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(dt_tor_data['Latitude'], dt_tor_data['Longitude'], dt_tor_data['Borough'], dt_tor_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dt_tor) 
    
    
map_dt_tor

Using the Foursquare API, we will use this Foursquares data for analysis as well.

In [47]:
CLIENT_ID = '5LLMMGIISKXPL2BQE11OVPL0IFFV0O2UD2EIMXK4YGY2JMMT' # your Foursquare ID
CLIENT_SECRET = 'AVGCI4XP2RW3NT2R0DNT4RTQR15BWW1ENLOL12FHRM4V0QN1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5LLMMGIISKXPL2BQE11OVPL0IFFV0O2UD2EIMXK4YGY2JMMT
CLIENT_SECRET:AVGCI4XP2RW3NT2R0DNT4RTQR15BWW1ENLOL12FHRM4V0QN1


Lets explore some neighbourhoods within Boroughs now, let us begin with Downtown Toronto.

In [51]:
dt_tor_data.loc[0:20]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


In [66]:

neigh_lat = dt_tor_data.loc[1, 'Latitude'] # neighborhood latitude value
neigh_lng = dt_tor_data.loc[1, 'Longitude'] # neighborhood longitude value

neigh_name = dt_tor_data.loc[1, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_lat, 
                                                               neigh_lng))

Latitude and longitude values of Queen's Park, Ontario Provincial Government are 43.6623015, -79.3894938.


In [68]:
LIMIT = 100 
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [69]:
results = requests.get(url).json()

We will use the function get_category_type (from IBM data science lab) to help structre the results data we just queried. 

In [72]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [98]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.location.address' ,'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# nearby_venues['venue.categories'][0]

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,address,categories,lat,lng
0,Queen's Park,University Ave.,Park,43.663946,-79.392180
1,Mercatto,101 College St,Italian Restaurant,43.660391,-79.387664
2,Neo Coffee Bar,770 Bay Street Unit 3,Coffee Shop,43.660140,-79.385870
3,T-Swirl Crepe,510 Yonge Street,Creperie,43.663452,-79.384125
4,Central YMCA,20 Grosvenor St,Distribution Center,43.663083,-79.385025


We can get nearby venues for all neighbourhoods in Downtown Toronto (from IBM data science lab).

In [100]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [101]:
dt_tor_venues = getNearbyVenues(names=dt_tor_data['Neighbourhood'],
                                   latitudes=dt_tor_data['Latitude'],
                                   longitudes=dt_tor_data['Longitude']
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


Next is a data frame of all the venues around the different neighbourhoods of Downtown Toronto.  

In [103]:
dt_tor_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [104]:
dt_tor_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,63,63,63,63,63,63
Christie,17,17,17,17,17,17
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


Now that we have this data cleaned and prepared we will move to anaylzing it. In order to do this we must create some dummy varaibles.

In [128]:
dt_tor_encod = pd.get_dummies(dt_tor_venues[['Venue Category']], prefix = '', prefix_sep = '')

# add neighborhood column back to dataframe
dt_tor_encod['Neighbourhood'] = dt_tor_venues['Neighbourhood'] 

# move neighborhood column to the first column
firstcol = dt_tor_encod.pop('Neighbourhood')
dt_tor_encod.insert(0, 'Neighbourhood', firstcol)

dt_tor_encod.head()


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [131]:
dt_tor_neigh_grouped = dt_tor_encod.groupby('Neighbourhood').mean().reset_index()

Now, we can find each neighbourhoods 3 most common venues.

In [133]:
top_venues = 3

for i in dt_tor_neigh_grouped['Neighbourhood']:
    print("----"+i+"----")
    temp = dt_tor_neigh_grouped[dt_tor_neigh_grouped['Neighbourhood'] == i].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2      Beer Bar  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1  Italian Restaurant  0.06
2      Sandwich Place  0.06


----Christie----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1  Japanese Restaurant  0.06
2     Sushi Restaurant  0.05


----Commerce Court, Victoria Hotel----
         venue  freq
0  Coffee Shop  0.11
1         Café  0.07
2   Restaurant  0.07


----First Canadian Place, Underground city----
         venue  freq
0  Coffee Shop  0.10
1         Café  0.08
2        Hotel  0.04


----Garden D

Finally, we can create a new data frame that will rank the top 10 venues per neighbourhood in Downtown Toronto.

In [134]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [138]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = dt_tor_neigh_grouped['Neighbourhood']

for ind in np.arange(dt_tor_neigh_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_tor_neigh_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Seafood Restaurant,Beer Bar,Restaurant,Pub,Bakery,Café,Irish Pub
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Sculpture Garden,Plane,Bar,Harbor / Marina,Airport Food Court
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Japanese Restaurant,Salad Place,Bubble Tea Shop,Ice Cream Shop,Park
3,Christie,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Diner,Restaurant,Baby Store,Athletics & Sports,Nightclub
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Café,Pub,Smoke Shop,Hotel,Yoga Studio
5,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Deli / Bodega,Japanese Restaurant,Italian Restaurant,Seafood Restaurant
6,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,Asian Restaurant,Seafood Restaurant,Steakhouse,Deli / Bodega
7,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Italian Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Restaurant,Diner
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout,Brewery,Sporting Goods Shop,Restaurant,Fried Chicken Joint,Italian Restaurant
9,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Dessert Shop,Bakery,Vietnamese Restaurant,Mexican Restaurant,Gaming Cafe,Grocery Store,Bar,Vegetarian / Vegan Restaurant


In [158]:
# set number of clusters
kclusters = 5

dt_tor_grouped_clustering = dt_tor_neigh_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 4, 3, 2, 2, 2, 2, 2, 2], dtype=int32)

In [159]:
# add clustering labels
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# dt_tor_merged = dt_tor_data

# # merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
# dt_tor_merged = dt_tor_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

dt_tor_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Farmers Market,Performing Arts Venue
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,Coffee Shop,Hobby Shop,Restaurant,Bank,Bar,Diner,Smoothie Shop,Discount Store,Café,Yoga Studio
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Café,Italian Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Restaurant,Diner
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Restaurant,Beer Bar,Gym,Hotel,Italian Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Cheese Shop,Seafood Restaurant,Beer Bar,Restaurant,Pub,Bakery,Café,Irish Pub


In [160]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_tor_merged['Latitude'], dt_tor_merged['Longitude'], dt_tor_merged['Neighbourhood'], dt_tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters